In [ ]:
import random
import torch
import matplotlib as plt
from graphviz import Digraph

In [1]:
class value:
    def __init__(self, data, children=() , sign="", label=""):
        self.data = data
        self._prev = set(children)
        self.sign = sign
        self.label = label
        self._backward = lambda: None
        self.grad = 0.0

    def __add__(self,other):
        other = value(other ) if not isinstance(other, value) else other

        out = value(self.data + other.data, (self,other), sign="+")

        def _backward():
            self.grad = 1.0 * out.grad
            other.grad = 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + (other)

    def __mul__(self, other):
        other = value(other) if not isinstance(other, value) else other
        out = value(self.data * other.datam, (self, other), sign="*")

        def _backward():
            self.grad = other.grad * out.grad
            other.grad = self.grad * out.grad
        out._backward = _backward

        return out

print(value(2.0).data)

2.0


In [ ]:
class neuron:
    def __init__(self, nin):
        self.weights = [value(random.uniform(-1,1)) for x in range(nin)]
        self.bias = value(random.uniform(-1,1))

    def __call__(self, x):
        act = sum(((wi*xi) for wi,xi in zip(self.weights, x)), self.bias)

        out = act.tanh()

        return out
    

class layer:
    def __init__(self, nins, nout):
        self.neurons = [neuron(nins) for _ in range(nout)]

    def __call__(self, x):
        outs = [f(x) for f in self.neurons]

        return outs[0] if len(outs) == 1 else outs
    
class MLP:
    def __init__(self, nins, nouts):
        size = [nins] + nouts
        self.layers = [layer(i, i+1) for i in range(len(size))]

    def __call__(self, x):
        for f in (self.layers):
            x = f(x)
        return x